# Inversion Constraints

The constraints transform an ill-posed problem into a well-posed problem. So, we formulate a constrained non-linear inversion by summing constraints to the misfit function $\Psi (\textbf{m})$. 

\begin{equation}
\Gamma (\textbf{m}) = \Psi (\textbf{m}) + \mu \sum\limits^{6}_{j=1}\alpha_{j}\varphi_{j}(\textbf{m})$
\end{equation}

where $\textbf{m}$ is the parameter vector, $\varphi_{j}$ is the $j$th constraint, $\alpha_{j}$ is the $j$-th is a positive scalar that defines the weight of the $j$-th constraint and $\mu$ is a positive scalar called regularization parameter. To solve this problem we will use the Levenberg-Marquadt method.

In this code, we impose six constraints to an Hessian matrix and parameter vector. The functions are tested in some special cases.

### Impor required library

In [33]:
import numpy as np
import polyprism_tests as tests

### Import my functions from an external file

In [34]:
import mag_polyprism_functions as mfun

### Input

In [35]:
M = 3 # number of vertices
L = 1 # number of prisms
P = L*(M+2) # number of parameters

In [36]:
alpha = 1.0 # weight

In [37]:
H = np.zeros((P,P)) # Hessian matrix

In [38]:
m = np.ones(P) # gradient of parameter vector

### Constraints

Here we describe the constraints and the result from its functions.

### Smoothness constraint on the adjacent radial distances within each prism $\varphi_{1}(\textbf{m})$

Smoothness constraint on the estimated radial distances within each prism. This constraint imposes that the $j$-th radial distance of the $k$-th prism $r^{k}_{j}$ must be close to the adjacent one $r^{k}_{j+1}$ within the same $k$-th prism. It prevents descontinuity on the estimated radial distances within each prism and imposes a cylindrical shape to the prisms. So, the squared Euclidian Norm of the difference between the adjacent radial distances must be minimun:

\begin{equation}
\varphi_{1}(\textbf{m}) = \sum\limits^{L}_{k=1}\left[\left(r^{k}_{M}-r^{k}_{1}\right)^2 + \sum\limits^{M-1}_{j=1}\left(r^{k}_{j}-r^{k}_{j+1}\right)^2\right]
\end{equation}

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{1}(\textbf{m}) = \textbf{m}^{T}\textbf{R}^{T}_{1}\textbf{R}_{1}\textbf{m}
\end{equation}

where

\begin{equation}
\textbf{R}_{1} = 
\begin{bmatrix}
\textbf{R}^{\sharp} & 0 & \cdots & 0 \\
0 & \textbf{R}^{\sharp} &  \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 &  \cdots & \textbf{R}^{\sharp} \\
\end{bmatrix}_{ML\times P}
\end{equation}

while

\begin{equation}
\textbf{R}^{\sharp} = 
\begin{bmatrix}
1 & -1 & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & 1 & -1 & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 1 & -1 & 0 & 0 \\
-1 & 0 & 0 & \cdots & 0 & 1 & 0 & 0 \\
\end{bmatrix}_{M(L-1)\times P}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{1}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{1}(\textbf{m}) = \textbf{R}^{T}_{1}\textbf{R}_{1}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{1}(\textbf{m}) = \textbf{R}^{T}_{1}\textbf{R}_{1}
\end{equation}

In [39]:
m1 = mfun.gradient_phi_1(M,L,m,alpha) # m1 = gradient input + gradient of phi1
H1 = mfun.Hessian_phi_1(M,L,H,alpha) # H1 = Hessian input + Hessian of phi1

In [40]:
print m

[ 0.  0.  0.  1.  1.]


In [41]:
print H

[[ 2. -1. -1.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [-1. -1.  2.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


In [42]:
tests.test_Hessian_phi_1()